In [83]:
# load the first simple geometry
import fiona
from shapely.geometry import Point,Polygon
c = fiona.open('shapefiles/cb_2013_us_ua10_500k.shp','r')
coordinates = []
for city in list(c):
    if city['geometry']['type'] == 'Polygon':
        # print(city)
        coordinates = city['geometry']['coordinates'][0]
        break
p = Polygon(coordinates)
print(p)
pt = Point([-97.58,27.94])
p.contains(pt)

POLYGON ((-110.642771 32.596403, -110.640142 32.599216, -110.641214 32.602232, -110.634806 32.609506, -110.632637 32.610398, -110.631214 32.612955, -110.631592 32.61415, -110.629253 32.615778, -110.631672 32.618789, -110.630124 32.619622, -110.628183 32.618931, -110.6286 32.617863, -110.626075 32.614449, -110.628104 32.614184, -110.617735 32.600968, -110.63069 32.599631, -110.632248 32.597527, -110.635779 32.59682, -110.637135 32.599611, -110.639453 32.597127, -110.638702 32.595627, -110.641772 32.594369, -110.642771 32.596403))


False

In [84]:
# load the first complex geometry
import fiona
from shapely.geometry import Point,Polygon,MultiPolygon
c = fiona.open('shapefiles/cb_2013_us_ua10_500k.shp','r')
coordinates = []
for city in list(c):
    if city['geometry']['type'] == 'MultiPolygon':
        # print(city)
        coordinates = city['geometry']['coordinates']
        break
# p = MultiPolygon(coordinates)
# print(p)

In [85]:
print(len(coordinates))
for c in coordinates:
    print(len(c))
    print(len(c[0]))
# there are 12, length one lists
# since the lists have length 1, I think this means
# that there are no holes
# and that these are 12 disjoint areas

12
1
18
1
35
1
8
1
13
1
16
1
28
1
6
1
7
1
8
1
1370
1
235
1
24


In [86]:
# The MultiPolygon constructor takes a sequence of exterior ring
# and hole list tuples: [((a1, ..., aM), [(b1, ..., bN), ...]), ...].
exterior_ring = ((1,1),(0,1),(0,0),(1,0))
interior_hole1 = ((.3,.3),(.1,3),(.1,.1),(.3,.1))
interior_hole2 = ((.9,.9),(.7,9),(.7,.7),(.9,.7))
p = MultiPolygon([(exterior_ring,[interior_hole1,interior_hole2]),(exterior_ring,[])])

In [87]:
try:
    p = MultiPolygon(coordinates)
except:
    print('nope')

nope


In [88]:
coordinates_w_empty_holes = [(tuple(c[0]),()) for c in coordinates]

In [89]:
p = MultiPolygon(coordinates_w_empty_holes)

In [90]:
pt = Point([-97.58,27.94])
p.contains(pt)

False

In [91]:
# load all of the geometries
import fiona
from shapely.geometry import Point,Polygon,MultiPolygon
c = fiona.open('shapefiles/cb_2013_us_ua10_500k.shp','r')
polygonList = []
nameList = []
polygonCount = 0
multiPolygonCount = 0
for city in list(c):
    nameList.append(city['properties']['NAME10'])
    if city['geometry']['type'] == 'Polygon':
        polygonCount += 1
        # print(city)
        coordinates = city['geometry']['coordinates']
        polygonList.append(Polygon(coordinates[0]))
    elif city['geometry']['type'] == 'MultiPolygon':
        multiPolygonCount += 1
        coordinates = city['geometry']['coordinates']
        # check that there aren't any holes:
        # for c in coordinates:
            # if len(c) != 1:
                # print('{0} has holes...in different area codes'.format(city['properties']['NAME10']))
                # print(c)
        coordinates_w_holes = [(tuple(c[0]),()) if len(c) == 1 else (tuple(c[0]),(c[1:])) for c in coordinates]
        polygonList.append(MultiPolygon(coordinates_w_holes))
    else:
        print('unknown geometry ' % city['geometry']['type'])
# p = MultiPolygon(coordinates)
# print(p)
print('done')

done


In [92]:
print(polygonCount)
print(multiPolygonCount)

1289
2312


In [93]:
pt = Point([-97.58,27.94])
for i,city in enumerate(polygonList):
    if city.contains(pt):
        print(i)
        print(nameList[i])
        break

22
Odem, TX


In [94]:
def cityID(polygonList,pt):
    for i,city in enumerate(polygonList):
        if city.contains(pt):
            return i
    return -1

In [95]:
locID = cityID(polygonList,pt)

In [96]:
import json
f = open('testfiles/test.json','r')
for line in f:
    tweet = json.loads(line)
    # print(tweet['coordinates']['coordinates'])
    pt = Point(tweet['coordinates']['coordinates'])
    loc = cityID(polygonList,pt)
    if loc > -1:
        print(nameList[loc])
    else:
        print('not found')

not found
not found
not found
not found
not found
Miami, FL
Lake Placid, NY
Boston, MA--NH--RI
not found
San Diego, CA
